# Resample and Save Locally
* using SimpleITK and numpy to preprocess each nifti image and save them in a local directory
    * resample -> normalize
* __Can possibly add bias correction and multiprocessing capabilities in?__

In [1]:
import numpy as np
import os
from shutil import copyfile
from io_func import resample_img, get_multi_class_labels, normalize_clip, to_channels_first, convert_4D_to_3D
from SimpleITK import GetImageFromArray, GetArrayFromImage, ReadImage
from glob import glob
from functools import partial
import time
#from pathos.multiprocessing import ProcessingPool as Pool
import multiprocessing as mp

In [2]:
def timeit(method):
    '''decorator for timing'''
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

In [5]:
class PreprocessLocal(object):
    '''Preprocesses 3D/4D nifti images and saves it locally compressed for the Medical Segmentation Decathlon Challenge for one task or a directory of images
    Static Methods:
    * create_new_dir: creates a new directory in the competition format 
    * preprocess_fn_and_save: applies a function to each image (nii or npz) in a specific directory, such as //imagesTr
    '''
    def __init__(self, data_path, output_path, num_classes=4):
        # specificies the path to //Task0_
        self.data_path = data_path
        self.output_path = output_path
        self.num_classes = num_classes
        # gets list of file names for saving the files; NOTE THAT filenames are the same for training images and labels
        for root, dirs, files in os.walk(self.data_path + '\\imagesTr\\'):
            filenames = [file[:-4] for file in files if file.endswith(".nii")]
        self.filenames = filenames
    
    @staticmethod
    def create_new_dir(output_path, data_path = None):
        '''
        Used to create a directory in the challenge format
        param output_path: path to new folder
        param data_path: path to original file location (task folder)
        '''
        os.mkdir(output_path)
        # 2 subdirectories
        os.mkdir(os.path.join(output_path, 'imagesTr'))
        os.mkdir(os.path.join(output_path, 'labelsTr'))
        if isinstance(data_path, (basestring, str, unicode)): 
            copyfile(data_path + '\\dataset.json', output_path +'\\dataset.json')
        print('Directory: ' + output_path + ' created.')
    
    @staticmethod
    def preprocess_fn_and_save(data_path, output_path, file_type, fn, fn_args):
        '''
        general function to preprocesses all files within a specific directory with a function that you specify
        ***NOTE: the data_path you need to specify here is different from the self.data_path one.
            must be a single specific directory, such as //imagesTr instead of the task folder
        param data_path: specific data directory to preprocess
        param output_path: specific directory to save the data
        param file_type: type of the files you want to preprocess (nii or npz)
        param fn: function you want to apply
        param fn_args: dictionary of arguments for the function
        '''
        if file_type == 'nii':
            for root, dirs, files in os.walk(data_path):
                filenames = [file[:-4] for file in files if file.endswith(".nii")]
            images_list = glob(data_path + '*.nii', recursive=True)
        elif file_type == 'npz':
            for root, dirs, files in os.walk(data_path):
                filenames = [file[:-4] for file in files if file.endswith(".npz")]
            images_list = glob(data_path + '*.npz', recursive=True)
        # checking to make sure there's a save directory
        if os.path.exists(output_path) == False:
            os.mkdir(output_path)
            print('Directory: ' + output_path + ' created.')
        # apply the function to each image in the directory and save them.
        for idx, image in enumerate(images_list):
        # need to save with file names
            np.savez_compressed(output_path + filenames[idx] , fn(image, file_type = file_type, **fn_args))
        print('Finished preprocessing and saving '+ str(len(images_list)) + ' files.')
        
    @timeit
    def preprocess(self, path, is_label = False):
        '''
        resamples nifti image to 1 mm isotropic spacing
        param path: path to a 3D/4D nifti1image image
        param is_label: Boolean representing whether or not the image is a label
        returns: a preprocessed numpy array
        '''
           
        sitk_read = ReadImage(path)
        # if label
        if is_label: 
            itk_label = GetArrayFromImage(resample_img(sitk_read, is_label=True))
            y = get_multi_class_labels(np.expand_dims(itk_label,1), self.num_classes)
            return y
        
        elif sitk_read.GetDimension() == 3:
            return normalize_clip(GetArrayFromImage(resample_img(sitk_read)))
                
        elif sitk_read.GetDimension() == 4:
            np_list = convert_4D_to_3D(sitk_read) 
            # resample each array
            resample_list = [GetArrayFromImage(resample_img(arr)) for arr in np_list]

            return normalize_clip(np.stack(resample_list, axis = 1))
        
    @timeit
    def preprocess_and_save(self):
        '''
        applies resampling and normalization of regular images and labels and saves the images.
        [original MSD Nifti1 images]
        ***Arguments are specified when initializing class.
        '''
        # gets lists of paths to images
        images_list = glob(self.data_path + '\\imagesTr\\*.nii', recursive=True)
        labels_list = glob(self.data_path + '\\labelsTr\\*.nii', recursive=True)
        # checking to make sure there's a save directory
        if os.path.exists(self.output_path) == False:
            PreprocessLocal.create_new_dir(self.output_path, self.data_path)
        # preprocess each image and save them.
        for (idx, image), label in zip(enumerate(images_list), labels_list):
        # need to save with file names
            np.savez_compressed(output_path + '\\imagesTr\\' + self.filenames[idx] , self.preprocess(image))
            np.savez_compressed(output_path + '\\labelsTr\\' + self.filenames[idx] , self.preprocess(label, is_label = True)) 
        print('Finished preprocessing and saving '+ str(len(images_list)) + ' files.')
        ############ WORKS IN PROGRESS
    ## SEPARATING preprocessing for multiprocessing
    def preprocess_regular(self):
        if os.path.exists(self.output_path) == False:
            PreprocessLocal.create_new_dir(self.output_path, self.data_path)
        images_list = glob(self.data_path + '\\imagesTr\\*.nii', recursive=True)
        for idx, image in enumerate(images_list):
        # need to save with file names
            np.savez_compressed(output_path + '\\imagesTr\\' + self.filenames[idx] , self.preprocess(image))
        print("Finished preprocessing " + str(len(images_list)) + ' files.')    
        
    def preprocess_labels(self):
        if os.path.exists(self.output_path) == False:
            PreprocessLocal.create_new_dir(self.output_path, self.data_path)
        labels_list = glob(self.data_path + '\\labelsTr\\*.nii', recursive=True)
        for idx, label in enumerate(labels_list):
        # need to save with file names
            np.savez_compressed(output_path + '\\labelsTr\\' + self.filenames[idx] , self.preprocess(label, is_label = True)) 
        print("Finished preprocessing " + str(len(labels_list)) + ' files.')    

    def run_processes(self):
        # I want each process to do the preprocessing for a separate image
        n_workers = mp.cpu_count()
        p_reg = mp.Process(target = self.preprocess_regular)
        p_lab = mp.Process(target = self.preprocess_labels )
        p_reg.start()
        p_lab.start()
#         print('Done!')
        

https://www.reddit.com/r/learnpython/comments/3ptclb/how_to_take_advantage_of_multiple_cores_inside_a/
<br> https://stackoverflow.com/questions/40097485/how-to-use-multiprocessing-in-python-within-a-class
* why doesn't it return shit in idle? **Test if it returns in python interpretor and check niftynet implementations.**

In [17]:
# #### preprocess_and_save()
# data_path = 'C:\\Users\\Joseph\\MSD\\MSD_raw\\Task02_Heart'#MSD_raw\\Task02_Heart
# output_path = 'C:\\Users\\Joseph\\MSD\\Heart_Preprocessed'
# saver = PreprocessLocal(data_path, output_path, num_classes = 2)
# %time saver.run_processes() 


Done!
Wall time: 300 ms


In [6]:
#### preprocess_and_save()
data_path = 'C:\\Users\\Joseph\\MSD\\MSD_raw\\Task02_Heart'#MSD_raw\\Task02_Heart
output_path = 'C:\\Users\\Joseph\\MSD\\Heart_Preprocessed'
saver = PreprocessLocal(data_path, output_path, num_classes = 2)
saver.preprocess_and_save()    

'preprocess'  11829.66 ms
'preprocess'  626.04 ms
'preprocess'  13049.73 ms
'preprocess'  509.03 ms
'preprocess'  12961.72 ms
'preprocess'  689.04 ms
'preprocess'  20252.13 ms
'preprocess'  728.04 ms
'preprocess'  10940.61 ms
'preprocess'  571.03 ms
'preprocess'  13505.75 ms
'preprocess'  667.04 ms
'preprocess'  13305.74 ms
'preprocess'  789.04 ms
'preprocess'  14826.82 ms
'preprocess'  1700.09 ms
'preprocess'  12555.70 ms
'preprocess'  780.04 ms
'preprocess'  20145.13 ms
'preprocess'  1035.06 ms
'preprocess'  21317.19 ms
'preprocess'  1084.06 ms
'preprocess'  17617.98 ms
'preprocess'  861.05 ms
'preprocess'  18788.05 ms
'preprocess'  780.05 ms
'preprocess'  14522.81 ms
'preprocess'  736.04 ms
'preprocess'  13912.78 ms
'preprocess'  784.04 ms
'preprocess'  12448.70 ms
'preprocess'  677.04 ms
'preprocess'  13642.76 ms
'preprocess'  704.04 ms
'preprocess'  13562.76 ms
'preprocess'  837.05 ms
'preprocess'  15232.85 ms
'preprocess'  672.04 ms
'preprocess'  12920.72 ms
'preprocess'  648.04 

In [ ]:
#### PreprocessLocal.preprocess_fn_and_save(...)
############# ADD FUNCTIONALITY TO ALLOW FOR CHANNELS FIRST DATA
d_path = 'C:\\Users\\Joseph\\MSD\\mexican'
o_path = 'C:\\Users\\Joseph\\MSD\\test'
from io_func import N4_bias_correction
def bias_corection(image_path, file_type, channels_format = 'channels_first', n_channels = 4):
    # loading files
    if file_type == 'nii':
        image = ReadImage(image_path)
    elif file_type == 'npz'
        image = GetImageFromArray(np.load(image_path)['array_0'].squeeze())
    # applying bias correction
    if image.GetDimension == 4: 
        np_list = convert_4D_to_3D(image) 
        # resample each array
        corrected_list = [GetArrayFromImage(N4_bias_correction(arr)) for arr in np_list]
        return np.stack(resample_list, axis = 1)
    if image.GetDimension == 3:
        return GetArrayFromImage(N4_bias_correction(image))
saver.preprocess_fn_and_save(d_path, o_path, )

# Testing

In [11]:
from io_func import resample_img, get_multi_class_labels, normalize_clip, to_channels_first, convert_4D_to_3D
from SimpleITK import GetImageFromArray, GetArrayFromImage, ReadImage
import unittest
import numpy as np
import SimpleITK as sitk

In [20]:
class TestLocalPreproccesingShapes(unittest.TestCase):
    '''
    tests the shapes for each step of the local preprocessing pipeline
    '''
    def setUp(self):
        base_path = 'C:\\Users\\Joseph\\MSD\\MSD_raw\\'
        self.data_path_4D = base_path + 'Task01_BrainTumour\\imagesTr\\BRATS_001.nii'
        self.data_path_3D = base_path + 'Task02_Heart\\imagesTr\\la_003.nii'
        self.label_path = base_path + 'Task01_BrainTumour\\labelsTr\\BRATS_001.nii'
        
    @unittest.skip("already works so skipping")      
    def test_preprocess_3D(self):
        '''
        tests to make sure that 3D preprocessing output shapes are correct
        '''
        sitk_read = ReadImage(self.data_path_3D)
        preprocessed = normalize_clip(GetArrayFromImage(resample_img(sitk_read)))
        self.assertEqual(preprocessed.shape, (178,400,400))
        
    @unittest.skip("already works so skipping")      
    def test_preprocess_4D(self):
        '''
        tests to make sure that 4D preprocessing output shapes are correct
        '''
        num_classes = 4
        sitk_read = ReadImage(self.data_path_4D)
        # converts to list of 3D arrays
        np_list = convert_4D_to_3D(sitk_read) 
        # resample each array
        resample_list = [GetArrayFromImage(resample_img(arr)) for arr in np_list]
        preprocessed = normalize_clip(np.stack(resample_list, axis = 1))
        self.assertEqual(preprocessed.shape, (155,4,240,240))
    
    @unittest.skip("already works so skipping")      
    def test_preprocess_labels(self):
        '''
        tests to make sure that label shapes are correct
        '''
        num_classes = 4
        sitk_read = ReadImage(self.label_path)
        itk_label = GetArrayFromImage(resample_img(sitk_read, is_label=True))
        y = get_multi_class_labels(np.expand_dims(itk_label,1), n_labels = num_classes)
        self.assertEqual(y.shape, (155,4,240,240))   
    
    def test_N4_bias_correction(self):
        '''
        tests the N4_bias_correction function from io_func
        '''
        # importing data and verifying dimensions + spacing
        d_path = 'C:\\Users\\Joseph\\MSD\\Heart_Preprocessed'
        arr = np.load(d_path + '\\imagesTr\\la_003.npz')['arr_0']
        itk_image = GetImageFromArray(arr)
        self.assertEqual(itk_image.GetSize(), (400,400,178))
        self.assertEqual(itk_image.GetSpacing(), (1.0, 1.0, 1.0))
        itk_image = sitk.Cast(itk_image,sitk.sitkFloat32)
        
        img_mask = sitk.Cast(sitk.BinaryNot(sitk.BinaryThreshold(itk_image, 0, 0)), sitk.sitkUInt8)
        self.assertEqual(img_mask.GetSize(), itk_image.GetSize())
        self.assertEqual(img_mask.GetSpacing(), itk_image.GetSpacing())
        self.assertEqual(img_mask.GetOrigin(), itk_image.GetOrigin())
        self.assertEqual(img_mask.GetDirection(), itk_image.GetDirection())
            
        bias_correction = sitk.N4BiasFieldCorrection(itk_image, img_mask)
        self.assertEqual(GetArrayFromImage(bias_correction).shape, (178,400,400))
        
    @unittest.skip('does not work')
    def test_transpose_output_and_N4_BiasCorrection(self):
        d_path = 'C:\\Users\\Joseph\\MSD\\Heart_Preprocessed'
        arr = np.load(d_path + '\\imagesTr\\la_003.npz')['arr_0']
        arr_labels = np.load(d_path + '\\labelsTr\\la_003.npz')['arr_0']
        self.assertEqual(np.transpose(np.expand_dims(arr,1), [1,0,2,3]).shape, (1,178,400,400))
        self.assertEqual(arr_labels.shape, (178, 2, 400,400))
        
        from io_func import N4_bias_correction
        itk_image = GetImageFromArray(arr)
        self.assertEqual(itk_image.GetSize(), (400,400,178))
        bias_correction = GetArrayFromImage(N4_bias_correction(itk_image))
        self.assertEqual(bias_correction.shape, (178,400,400))
                       
unittest.main(argv=[''], verbosity=2, exit=False)

test_N4_bias_correction (__main__.TestLocalPreproccesingShapes) ... ERROR
test_preprocess_3D (__main__.TestLocalPreproccesingShapes) ... skipped 'already works so skipping'
test_preprocess_4D (__main__.TestLocalPreproccesingShapes) ... skipped 'already works so skipping'
test_preprocess_labels (__main__.TestLocalPreproccesingShapes) ... skipped 'already works so skipping'
test_transpose_output_and_N4_BiasCorrection (__main__.TestLocalPreproccesingShapes) ... skipped 'does not work'

ERROR: test_N4_bias_correction (__main__.TestLocalPreproccesingShapes)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-20-6ae7fc094076>", line 63, in test_N4_bias_correction
    bias_correction = sitk.N4BiasFieldCorrection(itk_image, img_mask)
  File "C:\Users\Joseph\Miniconda3\envs\ml\lib\site-packages\SimpleITK\SimpleITK.py", line 51318, in N4BiasFieldCorrection
    return _SimpleITK.N4BiasFieldCorrection(*args)
RuntimeError: